# San Francisco crime data analysis

In this notebook, I used SQL for big data analysis on SF crime data. 
Data Source: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

In [3]:
from csv import reader
from pyspark.sql.functions import col, udf
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from ggplot import *
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [4]:
## Dowload data 
### https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD
#import urllib.request
#urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
#dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
#display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

In [5]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"
# use this file name later

In [6]:
# read data from the data storage
# please upload your data into databricks community at first. 
crime_data_lines = sc.textFile(data_path)

#prepare data 
#get rid of the quotation
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the first line of data
#display(crimes.take(1))

#get the total number of data 
print(crimes.count())


['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
2215024

In [7]:
from pyspark.sql import SparkSession
#Spark Session setting
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)

#display the df_out1 
#display(df_opt1)

#creat a tempview "sf_crime" for df_out1
df_opt1.createOrReplaceTempView("sf_crime")

## Abstract

(More details are illustrated in each sub question)

Generally speaking, my main goal here is to figure out the most serious crime problem in San francisco. I mainly used SQL query, insightful visualiztion  and outsource information to indicate the findings. The main crime problem I learned from the data in San francisco is the low-level crime and property crime such as burglary, larceny, shoplifting, and vandalism and for this, I illustrated three possible main business impacts that this phenomenon led on retail market field, high-tech development and tourism. What's more, advice and suggestions to the policy and police assignment are given based on the hour of the day and areas of San francisco.

###The number of crimes for different category

In [10]:
q1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)
display(q1_result)

category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


From the graph below, we know that Larceny/theft rank the 1st in terms of the number of crimes in San Francisco.

In [12]:
#Spark SQL based
crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)

category,Count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


###The number of crimes for different district

From the table below, Southern district has the most number of crime through out Jan 2013 to May 2018. And in Sourthern area, the LARCENY/THEFT has the proportion of 28% of the total number of crime.

In [15]:
Q2 = spark.sql("SELECT PdDistrict, count(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC")
display(Q2)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


Specifically, in the Southern area, larceny/theft is the highest number of crime category.

In [17]:
Q2_plus = spark.sql("SELECT Category, count(*) AS Count FROM sf_crime WHERE PdDistrict = 'SOUTHERN' GROUP BY Category ORDER BY Count DESC")
display(Q2_plus)

Category,Count
LARCENY/THEFT,113342
OTHER OFFENSES,51403
NON-CRIMINAL,49821
ASSAULT,31314
WARRANTS,22024
DRUG/NARCOTIC,20901
VANDALISM,17449
SUSPICIOUS OCC,13019
BURGLARY,12551
VEHICLE THEFT,11419


### San Francisco Downtoan Area

Here, I counted the nubmer of cimes each Sunday at San Francisco downtown. The longtitude and latitude of center SF downtown is (37.7945742, -122.3999445). I took a point that is at the edge of the boundary of SF downtown in google map and found the eucliean distance between this point and the center of SF downtown in terms of longtitude and latitude. If the region in the table has a distance that is larger than the max distance defined above, then it is considered as outside of SF downtown. 

From the graph below, there does not exist an obvious trend in terms of the number of crime on each Sunday in SF downtown. But notice that compared to the average number of crime in whole SF area, the average number of crime in SF downtown is much lower. It indicates that SF downtown is a relatively a safer area in Sunday.

In [20]:
max_y, max_x = 37.798063, -122.406757
cent_y, cent_x = 37.7945742, -122.3999445

Q3_df_opt1 = df_opt1.withColumn("X", df_opt1["X"].cast(FloatType())).withColumn("Y", df_opt1["Y"].cast(FloatType()))

if_sfdt = udf(lambda x, y: (x - cent_x)**2 + (y - cent_y)**2 < (max_x - cent_x)**2 + (max_y - cent_y)**2, BooleanType())

Q3_tmp = Q3_df_opt1.withColumn('if_df', if_sfdt('X','Y'))
Q3_tmp.createOrReplaceTempView("sf_crime_q3")
Q3 = spark.sql("SELECT Date, count(*) as Count FROM sf_crime_q3 WHERE if_df = true and DayOfWeek = 'Sunday' GROUP BY Date ORDER BY Date DESC")
display(Q3)

Date,Count
12/31/2017,17
12/31/2006,16
12/30/2012,9
12/30/2007,9
12/29/2013,6
12/28/2014,13
12/28/2008,8
12/28/2003,10
12/27/2015,16
12/27/2009,14


In [21]:
spark.sql("SELECT AVG(Count) AS Downtown FROM (SELECT count(*) as Count FROM sf_crime_q3 WHERE if_df = true and DayOfWeek = 'Sunday' GROUP BY Date ORDER BY Date DESC)").show()
spark.sql("SELECT AVG(Count) AS WholeArea FROM (SELECT count(*) as Count FROM sf_crime_q3 WHERE DayOfWeek = 'Sunday' GROUP BY Date ORDER BY Date DESC)").show()

+--------+
Downtown|
+--------+
13.54625|
+--------+

+-----------------+
 WholeArea|
+-----------------+
367.3216957605985|
+-----------------+

### The number of crime in each month of 2015, 2016, 2017, 2018

From the graph below, there is no big difference among 2015, 2016 and 2017. But we can see that in 2018 the total number of crime between Jan and Apr (The data in 2018 is not complete after May, so we only look at data between Jan and Apr) is less than the average of the other three year among Jan, Feb, Mar and Apr. After digging some information in the internet, I found out that San Francisco is the nation’s leader in property crime such as burglary, larceny, shoplifting, and vandalism. And from the graph below, LARCENY/THEFT has the largest proportion among all the category. 

Here, I will briefly illustrate three impacts that this phenomenon led.
The first one is the impact on retail market. Property and other supposedly low-level crimes are intensifying the destruction of the retail market. According to the article on City Jounal (https://www.city-journal.org/san-francisco-crime), Landmark Mission District stores are shuttering, citing theft and lack of security. In April 2019, CVS closed two pharmacies that had been ravaged by constant shoplifting. And the destruction of the retail market will also further lead to discouragment in community improvement in San francisco. 

The second one is the impact on high techonology improvement. San francisco is famous for the high techonolgy compnay and unicorn company like Uber, Lyft and Aribnb in Silicon Valley and San francisco area. The high incidence rate of low-level crime and low resolution rate (discussed in later question) may discourage the recuitment of high-tech people and expansion of high-tech company in San francisco. 

The third one is the impact on tourism. San francisco is a great place to travel especially in holiday. However, people may be more willing to choose a safer place to spend their holiday.

In [24]:
Q4_plus = spark.sql("SELECT year(NewDate) as Year, Category, count(*) as Count \
          FROM (SELECT Category, DATE_FORMAT(TO_DATE(Date, 'MM/dd/yyyy'), 'yyyy-MM-dd') as NewDate FROM sf_crime) \
          GROUP BY Year, Category \
          HAVING Year in (2015, 2016, 2017, 2018) \
          ORDER BY Year")
display(Q4_plus)

Year,Category,Count
2015,GAMBLING,29
2015,NON-CRIMINAL,19177
2015,PROSTITUTION,374
2015,BAD CHECKS,38
2015,WARRANTS,6815
2015,DRIVING UNDER THE INFLUENCE,430
2015,SUSPICIOUS OCC,5500
2015,VANDALISM,7675
2015,FORGERY/COUNTERFEITING,763
2015,VEHICLE THEFT,7943


In [25]:
Q4 = spark.sql("SELECT year(NewDate) as Year, month(NewDate) as Month, Category, count(*) as Count \
          FROM (SELECT Category, DATE_FORMAT(TO_DATE(Date, 'MM/dd/yyyy'), 'yyyy-MM-dd') as NewDate FROM sf_crime) \
          GROUP BY Year, Month, Category \
          HAVING Year in (2015, 2016, 2017, 2018) \
          ORDER BY Year, Month")
display(Q4)

Year,Month,Category,Count
2015,1,ARSON,41
2015,1,KIDNAPPING,40
2015,1,RECOVERED VEHICLE,73
2015,1,OTHER OFFENSES,1809
2015,1,GAMBLING,1
2015,1,ROBBERY,360
2015,1,FORGERY/COUNTERFEITING,79
2015,1,EMBEZZLEMENT,12
2015,1,DRIVING UNDER THE INFLUENCE,34
2015,1,STOLEN PROPERTY,82


### The number of crime w.r.t the hour in certian day

#### Thanksgiving Day
In this part, I chose 12/23 - 12/27 these five days in each 2015, 2016 and 2017 to analysis. First, compared with the average number of crime for each year, the average number of crime for these five days across Christmas is much lower. And across these five days, 12/25 (Chiristmas) had the least number of crime. And during a day, the trend will be stably low between 2am to 7am, then hit a spike at noon and then go down a bit from afternoon to night. Overall, I don't see a huge increase in number of crime during the Chiristmas holiday season. On the contrary, the data shows a good and safe sign during the time. So I would suggest that spending Christmas in San francisco is not a bad choice. Also, some certain area like Southern San fransico may need to be avoided due to high incidence of crime and further analysis based on the district needs to be done.

In [28]:
#For General:
spark.sql("SELECT year(NewDate) as Year, count(*)/365 as YearAve \
          FROM (SELECT DATE_FORMAT(TO_DATE(Date, 'MM/dd/yyyy'), 'yyyy-MM-dd') as NewDate FROM sf_crime) \
          GROUP BY Year \
          HAVING Year in (2015, 2016, 2017)\
          ORDER BY Year").show()

spark.sql("SELECT sum(Count) / 5 as DayAveChristmas FROM (SELECT year(NewDate) as Year, month(NewDate) as Month, \
          Day(NewDate) as Day, count(*) as Count \
          FROM (SELECT Time, DATE_FORMAT(TO_DATE(Date, 'MM/dd/yyyy'), 'yyyy-MM-dd') as NewDate FROM sf_crime) \
          GROUP BY Year, Month, Day \
          HAVING Year in (2015, 2016, 2017) and Month = 12 and abs(DAY - 25) < 3\
          ORDER BY Year, Month, Day) GROUP BY Year, Month").show()

+----+------------------+
Year| YearAve|
+----+------------------+
2015| 428.8383561643836|
2016|413.37808219178083|
2017|424.03561643835616|
+----+------------------+

+---------------+
DayAveChristmas|
+---------------+
 363.4|
 347.0|
 370.6|
+---------------+

#### Christmas

In [30]:
Q5_Ch = spark.sql("SELECT year(NewDate) as Year, month(NewDate) as Month, \
          Day(NewDate) as Day, count(*) as Count, SUBSTRING(Time, 1, 2) as Hour \
          FROM (SELECT Time, DATE_FORMAT(TO_DATE(Date, 'MM/dd/yyyy'), 'yyyy-MM-dd') as NewDate FROM sf_crime) \
          GROUP BY Year, Month, Day, Hour \
          HAVING Year in (2015, 2016, 2017) and Month = 12 and abs(DAY - 25) < 3\
          ORDER BY Year, Month, Day, Hour")
display(Q5_Ch)

Year,Month,Day,Count,Hour
2015,12,23,6,00
2015,12,23,12,01
2015,12,23,13,02
2015,12,23,2,03
2015,12,23,4,04
2015,12,23,4,05
2015,12,23,8,06
2015,12,23,14,07
2015,12,23,9,08
2015,12,23,6,09


### Top 3 Dangerous District

The top-3 danger districts are Southern, Mission, Northern. Based on the graph below, after 12pm, the number of crime in all these three areas increase untill 7pm.  I suggest that assign more police at this time interval to these three areas. Also, although the number of crime is relatively low in the midnight, considering the level of danger of these areas, police department should still assign relatively more officers to these areas compared with other areas.

In [33]:
Q6 = spark.sql("SELECT PdDistrict, Category, SUBSTRING(Time, 1, 2) as Hour, count(*) as Count FROM sf_crime WHERE PdDistrict in ('SOUTHERN', 'MISSION', 'NORTHERN') \
               GROUP BY PdDistrict, Category, Hour \
               ORDER BY Category, Hour")
display(Q6)

PdDistrict,Category,Hour,Count
NORTHERN,ARSON,00,18
SOUTHERN,ARSON,00,31
MISSION,ARSON,00,35
MISSION,ARSON,01,23
NORTHERN,ARSON,01,27
SOUTHERN,ARSON,01,27
SOUTHERN,ARSON,02,25
NORTHERN,ARSON,02,31
MISSION,ARSON,02,28
NORTHERN,ARSON,03,38


### Resolution and Suggestions

Since I mentioned in the Q4 that San Francisco has a high low-level crime rate, it's confirmed that vehicle theft, larceny/theft, Vandalism and Burglary take up a large portion of the total number of crime with respect to category. And the resolution rate for these four categories are really low (approximately 8% to 12%). So I think there should be more policy made to increase the resolution rate for these four categories such as assign more officers in high incidence area.

In [36]:
Q7 = spark.sql("SELECT table1.Category, sub, total, sub / total as percentage from (SELECT Category, count(case when Resolution != 'NONE' then 1 else null end) as sub FROM sf_crime GROUP BY Category) as table1 INNER JOIN (SELECT Category, count(*) as total FROM sf_crime GROUP BY Category) as table2 ON table1.Category = table2.Category ORDER BY percentage")
display(Q7)

Category,sub,total,percentage
RECOVERED VEHICLE,603,8716,0.06918311151904544
VEHICLE THEFT,10622,126602,0.08390072826653608
LARCENY/THEFT,42521,480448,0.08850281404022912
SUSPICIOUS OCC,9458,80444,0.11757247277609269
VANDALISM,14169,116059,0.12208445704340035
BURGLARY,14890,91543,0.16265580109893713
ARSON,757,3931,0.19257186466547951
BAD CHECKS,181,925,0.19567567567567568
EXTORTION,164,741,0.2213225371120108
NON-CRIMINAL,53465,238323,0.2243383978885798


In [37]:
display(Q7)

Category,sub,total,percentage
RECOVERED VEHICLE,603,8716,0.06918311151904544
VEHICLE THEFT,10622,126602,0.08390072826653608
LARCENY/THEFT,42521,480448,0.08850281404022912
SUSPICIOUS OCC,9458,80444,0.11757247277609269
VANDALISM,14169,116059,0.12208445704340035
BURGLARY,14890,91543,0.16265580109893713
ARSON,757,3931,0.19257186466547951
BAD CHECKS,181,925,0.19567567567567568
EXTORTION,164,741,0.2213225371120108
NON-CRIMINAL,53465,238323,0.2243383978885798
